In [27]:
from rich.console import Console
from rich_theme_manager import ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [30]:
import gdelt

# get a handle to the gdelt data
gd2 = gdelt.gdelt(version=2)

# get events for today
data = gd2.Search(['2025-08-27'],
    table="events",
    coverage=True
)

# save the results in a CSV file to save time on reruns
data.to_csv("data/gdelt_results.csv")

In [32]:
import pandas as pd

# load the gdelt data
data = (
    pd
    .read_csv('data/gdelt_results.csv')
    .query('CAMEOCodeDescription.notna()')
    .reset_index(drop=True)
    .to_dict('records')
)

# output a couple samples
console.print(data[:2])

[
    {
        'Unnamed: 0': 0,
        'GLOBALEVENTID': 1259740626,
        'SQLDATE': 20240827,
        'MonthYear': 202408,
        'Year': 2024,
        'FractionDate': 2024.6493,
        'Actor1Code': 'FRA',
        'Actor1Name': 'FRANCE',
        'Actor1CountryCode': 'FRA',
        'Actor1KnownGroupCode': nan,
        'Actor1EthnicCode': nan,
        'Actor1Religion1Code': nan,
        'Actor1Religion2Code': nan,
        'Actor1Type1Code': nan,
        'Actor1Type2Code': nan,
        'Actor1Type3Code': nan,
        'Actor2Code': nan,
        'Actor2Name': nan,
        'Actor2CountryCode': nan,
        'Actor2KnownGroupCode': nan,
        'Actor2EthnicCode': nan,
        'Actor2Religion1Code': nan,
        'Actor2Religion2Code': nan,
        'Actor2Type1Code': nan,
        'Actor2Type2Code': nan,
        'Actor2Type3Code': nan,
        'IsRootEvent': 1,
        'EventCode': 193,
        'CAMEOCodeDescription': ' Fight with small arms and light weapons',
        'EventBaseCode': 193,
        'EventRootCode': 19,
        'QuadClass': 4,
        'GoldsteinScale': -10.0,
        'NumMentions': 1,
        'NumSources': 1,
        'NumArticles': 1,
        'AvgTone': 1.83098591549296,
        'Actor1Geo_Type': 1,
        'Actor1Geo_FullName': 'Spain',
        'Actor1Geo_CountryCode': 'SP',
        'Actor1Geo_ADM1Code': 'SP',
        'Actor1Geo_ADM2Code': nan,
        'Actor1Geo_Lat': 40.0,
        'Actor1Geo_Long': -4.0,
        'Actor1Geo_FeatureID': 'SP',
        'Actor2Geo_Type': 0,
        'Actor2Geo_FullName': nan,
        'Actor2Geo_CountryCode': nan,
        'Actor2Geo_ADM1Code': nan,
        'Actor2Geo_ADM2Code': nan,
        'Actor2Geo_Lat': nan,
        'Actor2Geo_Long': nan,
        'Actor2Geo_FeatureID': nan,
        'ActionGeo_Type': 2,
        'ActionGeo_FullName': 'New York, United States',
        'ActionGeo_CountryCode': 'US',
        'ActionGeo_ADM1Code': 'USNY',
        'ActionGeo_ADM2Code': nan,
        'ActionGeo_Lat': 42.1497,
        'ActionGeo_Long': -74.9384,
        'ActionGeo_FeatureID': 'NY',
        'DATEADDED': 20250827013000,
        'SOURCEURL': 
'https://www.dailymail.co.uk/news/royals/article-15036811/JAN-MOIR-Meghan-second-helping-woman-boasting-hosting-mak
es-curl-edges-like-squid-hot-grill.html'
    },
    {
        'Unnamed: 0': 1,
        'GLOBALEVENTID': 1259740627,
        'SQLDATE': 20240827,
        'MonthYear': 202408,
        'Year': 2024,
        'FractionDate': 2024.6493,
        'Actor1Code': 'MEX',
        'Actor1Name': 'MEXICO',
        'Actor1CountryCode': 'MEX',
        'Actor1KnownGroupCode': nan,
        'Actor1EthnicCode': nan,
        'Actor1Religion1Code': nan,
        'Actor1Religion2Code': nan,
        'Actor1Type1Code': nan,
        'Actor1Type2Code': nan,
        'Actor1Type3Code': nan,
        'Actor2Code': 'USA',
        'Actor2Name': 'UNITED STATES',
        'Actor2CountryCode': 'USA',
        'Actor2KnownGroupCode': nan,
        'Actor2EthnicCode': nan,
        'Actor2Religion1Code': nan,
        'Actor2Religion2Code': nan,
        'Actor2Type1Code': nan,
        'Actor2Type2Code': nan,
        'Actor2Type3Code': nan,
        'IsRootEvent': 1,
        'EventCode': 10,
        'CAMEOCodeDescription': ' Make statement, not specified below',
        'EventBaseCode': 10,
        'EventRootCode': 1,
        'QuadClass': 1,
        'GoldsteinScale': 0.0,
        'NumMentions': 2,
        'NumSources': 1,
        'NumArticles': 2,
        'AvgTone': 2.38693467336684,
        'Actor1Geo_Type': 2,
        'Actor1Geo_FullName': 'Idaho, United States',
        'Actor1Geo_CountryCode': 'US',
        'Actor1Geo_ADM1Code': 'USID',
        'Actor1Geo_ADM2Code': nan,
        'Actor1Geo_Lat': 44.2394,
        'Actor1Geo_Long': -114.51,
        'Actor1Geo_FeatureID': 'ID',
        'Actor2Geo_Type': 2,
        'Actor2Geo_FullName': 'New York, United States',
        'Actor2Geo_CountryCode': 'US',
        'Actor2Geo_ADM1Code': 'USNY',
        'Actor2Geo_ADM2Code': nan,
        'Acto

In [5]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# create the embedding encoder LLM
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
# name the collection
collection_name="gdelt"

# create the collection to store the gdelt data
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [7]:
from codetiming import Timer

with Timer(name='qdrant_upload', text="GDELT data points uploaded in {:.2f}s"):
    # load the vector DB
    qdrant.upload_points(
        collection_name=collection_name,
        points=[
            models.PointStruct(
                id=idx,
                vector=encoder.encode(doc["CAMEOCodeDescription"]).tolist(),
                payload=doc
            ) for idx, doc in enumerate(data)
        ]
    )

GDELT data points uploaded in 1628.30s


In [8]:
# show some stats on the qdrant collection
console.print(qdrant.get_collection(collection_name=collection_name))

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=None,
    indexed_vectors_count=0,
    points_count=136879,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None,
        strict_mode_config=None
    ),
    payload_schema={}
)

In [12]:
# create a question
user_prompt = "who is making endorsements today?"

# get the vectors for the question
query_vector = encoder.encode(user_prompt).tolist()

# get the results
hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [29]:
def format_qdrant_results(results):
    """
    format the results

    :param results:
    :return:
    """
    formatted = []

    # for each result
    for res in results:
        # create a dict of the results
        item = {
            'id': res.id,
            'score': round(res.score, 3),
            'title': res.payload.get('ActionGeo_FullName', 'N/A'),
            'category': res.payload.get('CAMEOCodeDescription', 'N/A'),
            'url': res.payload.get('SOURCEURL', 'N/A'),
            'Actor1Geo_Lat': res.payload.get('Actor1Geo_Lat', 'N/A'),
            'Actor1Geo_Long': res.payload.get('Actor1Geo_Long', 'N/A'),
            'Actor2Geo_Lat': res.payload.get('Actor2Geo_Lat', 'N/A'),
            'Actor2Geo_Long': res.payload.get('Actor2Geo_Long', 'N/A'),
        }

        # add the formatted data to the list
        formatted.append(item)

    # return to the caller
    return formatted

# get the formatted results
formatted_results = format_qdrant_results(hits)

console.print('Prompt:', user_prompt)

# display the results
for item in formatted_results:
    console.print(f"ID: {item['id']}, Score: {item['score']}, Title: {item['title']}, Category: {item['category']}, actor 1 lat/long: {item['Actor1Geo_Lat']},{item['Actor1Geo_Long']}, actor 2 lat/long: {item['Actor2Geo_Lat']},{item['Actor2Geo_Long']},[link={item['url']}]Link to the reference[/link]")


Prompt: who is making endorsements today?

ID: 136770, Score: 0.44, Title: Alaska, United States, Category:  Rally support on behalf of, actor 1 lat/long: 
61.385,-152.268, actor 2 lat/long: 49.0,32.0,]8;id=625029;https://alaskapublic.org/news/politics/2025-08-14/12-things-to-know-about-the-historic-trump-putin-meeting-in-anchorage\Link to the reference]8;;\

ID: 136769, Score: 0.44, Title: Ukraine, Category:  Rally support on behalf of, actor 1 lat/long: 61.2181,-149.9, 
actor 2 lat/long: 49.0,32.0,]8;id=31471;https://alaskapublic.org/news/politics/2025-08-14/12-things-to-know-about-the-historic-trump-putin-meeting-in-anchorage\Link to the reference]8;;\

ID: 57314, Score: 0.44, Title: White House, District of Columbia, United States, Category:  Rally support on behalf
of, actor 1 lat/long: nan,nan, actor 2 lat/long: 38.8951,-77.0364,]8;id=209532;https://www.yahoo.com/news/articles/cnn-pundit-fitness-guru-accused-100154476.html\Link to the reference]8;;\